In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# ----------------------------
# 0. Path file input/output
# ----------------------------
elements_csv = "D:/…/orbit_lines_and_elements_elements.csv"
output_csv   = "D:/…/one_revolution_paths.csv"

# ----------------------------
# 1. Kepler & Propagasi
# ----------------------------
def solve_kepler_elliptic(M, e, tol=1e-9, max_iter=100):
    E = M
    for _ in range(max_iter):
        f       = E - e*np.sin(E) - M
        f_prime = 1 - e*np.cos(E)
        delta   = -f/f_prime
        E      += delta
        if abs(delta) < tol:
            return E
    return M  # fallback

def elements_to_rv(a, e, i, raan, argp, M0, t0, mu, t):
    # Mean motion
    n  = np.sqrt(mu / a**3)
    M  = (M0 + n*(t - t0)) % (2*np.pi)
    E  = solve_kepler_elliptic(M, e)
    # Perifocal coords
    x_p   = a*(np.cos(E)-e)
    y_p   = a*np.sqrt(1-e**2)*np.sin(E)
    # Rotation to ECI
    cosO, sinO = np.cos(raan), np.sin(raan)
    cosw, sinw = np.cos(argp), np.sin(argp)
    cosi, sini = np.cos(i), np.sin(i)
    P = np.array([
      cosO*cosw - sinO*sinw*cosi,
      sinO*cosw + cosO*sinw*cosi,
      sinw*sini
    ])
    Q = np.array([
     -cosO*sinw - sinO*cosw*cosi,
     -sinO*sinw + cosO*cosw*cosi,
      cosw*sini
    ])
    return x_p*P + y_p*Q  # r_vec in km

# ----------------------------
# 2. Baca elemen orbital
# ----------------------------
df_el = pd.read_csv(elements_csv)

# Constants & conversions
mu_sun = 1.32712440018e11  # km^3/s^2
AU2km   = 1.496e+8

# ----------------------------
# 3. Generate one rev per objek
# ----------------------------
paths = []
num_points = 200

for _, row in df_el.iterrows():
    name = row['name']
    # Ambil & konversi
    a    = float(row['sma'])  * AU2km
    e    = float(row['ecc'])
    i    = float(row['inc'])
    raan = float(row['raan'])
    argp = float(row['argp'])
    M0   = float(row['M0'])
    t0   = 0.0               # kita set epoch t0 = 0
    mu   = mu_sun

    # Hitung periode T
    n = np.sqrt(mu / a**3)
    T = 2 * np.pi / n

    # Waktu diskrit 0 → T (exclude titik akhir ganda)
    times = np.linspace(0, T, num_points+1)[:-1]

    for tt in times:
        r_km = elements_to_rv(a,e,i,raan,argp,M0,t0,mu,tt)
        paths.append({
            'name': name,
            'time_offset_s': tt,
            'x_au': r_km[0]/AU2km,
            'y_au': r_km[1]/AU2km,
            'z_au': r_km[2]/AU2km
        })

# ----------------------------
# 4. Simpan CSV
# ----------------------------
pd.DataFrame(paths).to_csv(output_csv, index=False)
print(f"Selesai: satu revolusi per objek disimpan di {output_csv}")
